In [1]:
from collections import deque
import numpy as np
import theano
import theano.tensor as T
import lasagne

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [42]:
def build_qnn(input_var=None, input_shape=None, num_outputs=None):
    if input_shape is None or num_outputs is None:
        raise ValueError('State or Action dimension not given!')
    l_in = lasagne.layers.InputLayer(shape=input_shape, input_var=input_var)
    l_hid = lasagne.layers.DenseLayer(
        l_in, num_units=500, 
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    l_out = lasagne.layers.DenseLayer(
        l_hid, num_units=num_outputs,
        nonlinearity=lasagne.nonlinearities.sigmoid)
    return l_out

In [2]:
data_buffer = deque()

In [ ]:
FREEZE_INTERVALS = 100
count_step = 0

old_states = np.float32(np.random.rand(BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1]))
actions = np.int32(np.random.randint(0, 3, (BATCH_SIZE,)))
rewards = np.float32(np.random.rand(BATCH_SIZE,))
new_states = np.float32(np.random.rand(BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1]))

data_buffer = deque()

while(True):
    # Buffer trainning samples by using the target network
    next_state, reward
    current_state, current_action
    data_buffer.append((current_state, current_action, reward, next_state))
    _ = data_buffer.popleft() if len(data_buffer) > (FREEZE_INTERVALS+MEMORY_LENGTH) else None    
    if mod(FREEZE_INTERVALS, count_step)==0:
        # form trainning data
        
        # Train q network and substitute target network
        
        # clear cache       
        count_step = 0
    count_step += 1
        


In [95]:
DIM_STATE = [2, 3]
NUM_ACTIONS = 4

BATCH_SIZE = 1
MEMORY_LENGTH = 1
LEARNING_RATE = 0.1
GAMMA = 0.1

# inputs
old_states = T.tensor4('old_states')     # (BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1])
actions = T.ivector('actions')           # (BATCH_SIZE, 1)
rewards = T.vector('rewards')            # (BATCH_SIZE, 1)
new_states = T.tensor4('new_states')     # (BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1])
# intemediate
network = build_qnn(None, input_shape=(None, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1]), num_outputs=NUM_ACTIONS)
predict_q = lasagne.layers.get_output(layer_or_layers=network, inputs=old_states)
predict_next_q= lasagne.layers.get_output(layer_or_layers=network, inputs=new_states)
target_q = rewards + GAMMA*T.max(predict_next_q, axis=1)
# outputs
loss = T.mean((predict_q[T.arange(BATCH_SIZE), actions] - target_q)**2)
# weight updates
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.sgd(loss, params, learning_rate=LEARNING_RATE)
# functions
f_train_batch = theano.function([old_states, actions, rewards, new_states], loss, updates=updates)  # training function for one batch
f_get_q = theano.function([old_states], predict_q)

In [96]:
# Simple test
trn_old_states = np.float32(np.random.rand(BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1]))
trn_actions = np.int32(np.random.randint(0, 3, (BATCH_SIZE,)))
trn_rewards = np.float32(np.random.rand(BATCH_SIZE,))
trn_new_states = np.float32(np.random.rand(BATCH_SIZE, MEMORY_LENGTH, DIM_STATE[0], DIM_STATE[1]))
for i in range(100):
    print f_train_batch(trn_old_states, trn_actions, trn_rewards, trn_new_states),
    print "   "
    print f_get_q(trn_old_states)
    

0.128790944815    
[[ 0.55402976  0.51630974  0.4835743   0.53040069]]
0.114938691258    
[[ 0.57377851  0.51733088  0.48316655  0.53145164]]
0.102828301489    
[[ 0.59201956  0.51830411  0.48285705  0.53249836]]
0.0922465249896    
[[ 0.6089083   0.51916397  0.48256528  0.53354549]]
0.0829684734344    
[[ 0.62441236  0.51989192  0.48236698  0.53464425]]
0.0749006643891    
[[ 0.63874233  0.5205639   0.48219314  0.53571939]]
0.0678152069449    
[[ 0.65189195  0.52118003  0.48211899  0.5367344 ]]
0.0616295449436    
[[ 0.66397429  0.52172464  0.48208174  0.53774273]]
0.0562158748507    
[[ 0.67515737  0.52221924  0.4820199   0.53868961]]
0.051431979984    
[[ 0.68554276  0.52266085  0.48195899  0.53952265]]
0.0471815280616    
[[ 0.69523793  0.5230794   0.48191091  0.54028422]]
0.0433817133307    
[[ 0.70428693  0.52348888  0.48186198  0.54100889]]
0.0399805679917    
[[ 0.71265745  0.52386361  0.48180833  0.54163295]]
0.0369622893631    
[[ 0.72049642  0.52423304  0.48176262  0.5422281

0.0472981147468
0.0395744778216
0.0333338342607
0.0283946730196
0.0245178695768
0.0214814655483
0.0190833546221
0.0171810984612
0.0156597718596
0.0144281964749
0.0134187508374
0.0125743299723
0.0118695329875
0.0112834405154
0.0107846492901
0.0103502571583
0.00996846519411
0.00963180325925
0.00933013297617
0.00905758608133
0.00880928523839
0.00858221855015
0.00837110448629
0.00817354768515
0.00798865035176
0.00781634449959
0.00765094999224
0.00749426707625
0.00734523311257
0.00720066297799
0.00706561747938
0.00693342229351
0.00680606532842
0.006684652064
0.00656394660473
0.00644958531484
0.0063367811963
0.00622710073367
0.00612077256665
0.00601606955752
0.00591573631391
0.00581551110372
0.00571949034929
0.00562543608248
0.00553229171783
0.00544388080016
0.0053551835008
0.00526830088347
0.00518487021327
0.0051010325551
0.00502001820132
0.00494105089456
0.00486214924604
0.00478599453345
0.00471188127995
0.00463777966797
0.00456569669768
0.00449610501528
0.00442625209689
0.00435776682571
0